In [2]:
import pandas as pd
import numpy as np
import re

In [3]:
r_c=pd.read_excel('recipe_from_CHECKSCM_20240125.xlsx')
r_c.drop('Cost Centre', axis=1, inplace=True)
r_c.drop_duplicates(inplace=True)

In [4]:
r_c['Product']=r_c['Product'].astype(str)
r_c['Recipe Code']=r_c['Recipe Code'].astype(str).str.zfill(10)
r_c['Product']=r_c['Product'].astype(str).str.zfill(10)
rec_comp = r_c[['Product', 'Product Name']].merge(r_c[['Recipe Code', 'Recipe Name']], how='inner', left_on='Product', right_on='Recipe Code')
mapping_dict = r_c.set_index('Recipe Code')['Recipe Name'].to_dict()
r_c['Product Name'] = r_c['Product Name'].fillna(r_c['Product'].map(mapping_dict))

In [5]:
r_c=r_c[['Product Name', 'Size','Unit Cost']]

In [6]:
pd.set_option('max_colwidth', None)

In [7]:
r_c.head()

,Product Name,Size,Unit Cost
0,OIL OLIVE POMACE PET,BTL5LTR,29.800
1,BUTTER PASTEURISED UNSALTED BLOCK 82% X 2.5KG 103831 ELLE ET VIRE (MOQ: 10KGS) KGS,KGS,20.700
2,HERBSPICEDRY POWDER PEPPER WHITE,PKT500G,5.000
3,SALT TABLE FINE 500GM,PKT500G,0.550
4,VEGFRESH ASPARAGUS AUST,KG,13.549


In [8]:
uni=r_c['Size'].unique()

In [9]:
uni=uni.tolist()

In [10]:
print(uni)

['BTL5LTR', 'KGS', 'PKT500G', 'KG', 'PKT2KG', 'PKT1LTR', 'PKT50G', 'PKT1KG', 'PKT', 'PKT2LTR', 'PCS', 'GM', 'PORTION', 'PKT150G', 'TIN2.55KG', 'TIN', 'PUN', 'BTL1.8LTR', 'BTL1.8L', 'BTL1.8LT', 'CTN', 'PKT5KG', 'PKT300G', 'BTL600ML', 'BAG2KG', 'BTL1KG', 'BTL640ML', 'BTL510G', 'TIN17KG', 'PKT250G', 'PKT100PCS', 'PKT3KG', 'BTL1.2KG', 'PKT20G', 'BTL', 'BTL1LTR', 'BTL935G', 'BOX100PCS', 'BAG50KG', 'TUB', 'BTL750ML', 'PKT12PCS', 'TIN454G', 'PKT3.23KG', 'PKT1LT', 'BOX100PC', 'TUB5LTR', 'BOX50PCS', 'TUB5LT', 'TUB3KG', 'BTL500ML', 'PAX', 'CTN8PKT', 'PKT100G', 'BTL750G', 'PUN160G', 'PUN170G', 'JAR', 'BTL700G', 'PKT682G', 'PKT20PCS', 'BTL320G', 'BOX2.5KG', 'BTL350G', 'LOAF', 'BTL400G', 'CTN6PKT', 'TUB400ML', 'BTL450G', 'PKT10PCS', 'TUB2.2KG', 'BTL840ML', 'BTL500G', 'BTL290ML', 'BAG100PKT', 'JAR100G', 'TUB500G', 'PC', 'TIN2KG', 'CTN12KG', 'CTN4PKT', 'TUB2KG', 'BTL200G', 'TUB2.5KG', 'PKT200G', 'BOX', 'BTL550ML', 'PKT25G', 'PKT450G', 'BAG25KG', 'CTN100PCS', 'TUB1KG', 'TUB3LT', 'TIN500G', 'BOX45PCS',

In [11]:
import pandas as pd

def calculate_cost_per_unit(df):
    # Définir les équivalences d'unités
    equivalences_unites = {
        'GM': 'G',
        'PCS':'PC',
        'LT': 'L',
        'LTR': 'L',
        'LITRE': 'L',
        'O':'OZ',
        'K':'KG'
    }

    # Expressions régulières pour extraire le poids et l'unité
    regex_poids = r'(\d*\.?\d+)'
    regex_unite = r'([A-Za-z]+)$'

    # Extraire le poids et l'unité
    df['qant'] = df['Size'].str.extract(regex_poids, expand=False).astype(float)
    df['Unite'] = df['Size'].str.split(regex_unite).str[1].str.upper()

    # Traiter les unités équivalentes
    df['Unite'] = df['Unite'].replace(equivalences_unites)

    # Si le size est en "pcs", extraire le nombre de pièces et calculer le coût par pièce
    df['Pieces'] = df['Size'].str.extract(r'(\d+)PCS', expand=False)
    df['Pieces'] = pd.to_numeric(df['Pieces'], errors='coerce')  # Convertir en nombre, NaN si pas de correspondance
    df['Pieces'] = df['Pieces'].fillna(1)  # Remplacer les NaN par 1 pour les tailles sans "pcs"

    # Si le poids est NaN, calculer le coût par l'unité donnée (comme "PKT" ou "BAG")
    df['qant'] = df['qant'].fillna(1)

    return df

# Appel de la fonction pour calculer le coût par unité
test = calculate_cost_per_unit(r_c)


In [12]:
test.head(5)

,Product Name,Size,Unit Cost,qant,Unite,Pieces
0,OIL OLIVE POMACE PET,BTL5LTR,29.800,5.0,L,1.0
1,BUTTER PASTEURISED UNSALTED BLOCK 82% X 2.5KG 103831 ELLE ET VIRE (MOQ: 10KGS) KGS,KGS,20.700,1.0,KGS,1.0
2,HERBSPICEDRY POWDER PEPPER WHITE,PKT500G,5.000,500.0,G,1.0
3,SALT TABLE FINE 500GM,PKT500G,0.550,500.0,G,1.0
4,VEGFRESH ASPARAGUS AUST,KG,13.549,1.0,KG,1.0


In [16]:
def convert_quantity(row):
    unit = row['Unite']
    quantity = row['qant']
    product_name = row['Product Name']
    if unit=='G':
        return quantity / 1000
    elif unit=='ML':
        return quantity / 1000
    elif unit=='OZ':
        return quantity * 0.02835
    elif unit=='LB':
        return quantity * 0.4536
    elif unit=='KGS':
        # Chercher la chaîne 'XKG' dans le nom du produit et extraire X
        match_xkg = re.search(r'(\d+(\.\d+)?)KG', product_name)
        if match_xkg:
            multiplier = float(match_xkg.group(1))
            return multiplier
        else:
            # Chercher la chaîne 'Y PCS' ou 'Y PC' et 'X G' ou 'X GM' dans le nom du produit et extraire X et Y
            match_y_pcs = re.search(r'(\d+(\.\d+)?) ?(PCS|PC)', product_name)
            match_x_g = re.search(r'(\d+(\.\d+)?)(G|GM)', product_name)
            if match_y_pcs and match_x_g:
                y_multiplier = float(match_y_pcs.group(1))
                x_multiplier = float(match_x_g.group(1))
                return (x_multiplier * y_multiplier) / 1000
            else:
                return ''
    elif unit=='KG':
        return quantity
    elif unit=='L':
        return quantity
    
    else:
        return ''
"""elif unit=='BAG':
        match_xkg = re.search(r'(\d+(\.\d+)?)KG', product_name)
        if match_xkg:
            multiplier = float(match_xkg.group(1))
            return multiplier
        else:"""
# Appliquer la fonction de conversion à la colonne 'qant'
test['KG_L'] = test.apply(convert_quantity, axis=1)
# Convert columns to numeric type
test['Unit Cost'] = pd.to_numeric(test['Unit Cost'], errors='coerce')
test['KG_L'] = pd.to_numeric(test['KG_L'], errors='coerce')

# Perform division
test['cout/KG_L'] = test['Unit Cost'] / test['KG_L']


In [17]:
test.head()

,Product Name,Size,Unit Cost,qant,Unite,Pieces,KG_L,cout/KG_L
0,OIL OLIVE POMACE PET,BTL5LTR,29.800,5.0,L,1.0,5.0,5.960
1,BUTTER PASTEURISED UNSALTED BLOCK 82% X 2.5KG 103831 ELLE ET VIRE (MOQ: 10KGS) KGS,KGS,20.700,1.0,KGS,1.0,2.5,8.280
2,HERBSPICEDRY POWDER PEPPER WHITE,PKT500G,5.000,500.0,G,1.0,0.5,10.000
3,SALT TABLE FINE 500GM,PKT500G,0.550,500.0,G,1.0,0.5,1.100
4,VEGFRESH ASPARAGUS AUST,KG,13.549,1.0,KG,1.0,1.0,13.549


In [19]:
test.to_excel("output_test_unit.xlsx", index=False)